In [57]:
## Reads excel data sheet and returns relevent data in a dictionary ##
## Returns: stats_dictionary = {date : opening_price} ##

import csv
from collections import OrderedDict

def get_stats_from_excel(path, rev_sheet=False):
    stats_dictionary = OrderedDict()
    First = True

    with open(path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if First:
                First = False # skip the title row
                continue
        
            stats_dictionary[row[0]] = float(row[1])
        
        if rev_sheet:
            stats_dictionary = OrderedDict(reversed(list(stats_dictionary.items())))
            
        return(stats_dictionary)

In [58]:
## MAIN TRADING MECHANICS ##

# Buy a number of shares for a specified price
def buy(shares_bought, price, account):
    cost = shares_bought*price + trade_fee
    
    if(account['cash'] >= cost):
        account['cash'] -= cost
        account['shares'] += shares_bought
        account['receipts'].append((-1*cost, shares_bought))
    else:
        print('Not enough cash!')
    
    
# Sell a number of shares for a specified price
def sell(shares_sold, price, account):
    cash = shares_sold*price - trade_fee
    
    if(account['shares'] >= shares_sold):
        account['cash'] += cash
        account['shares'] -= shares_sold
        account['receipts'].append((cash, -1*shares_sold))
    else:
        print('Not enough shares!')
        
        
# Buy shares by percentage of cash in account
def buy_shares_by_percent_funds_owned(percent, current_cost, account):
    spendable = account['cash']*percent - trade_fee
    num_shares = int(spendable/current_cost)
    buy(num_shares, current_cost, account) 
    
    
# Sell shares by percentage of cash in account
def sell_shares_by_percent_shares_owned(percent, current_cost, account):
    num_shares = account['shares'] * percent
    sell(num_shares, current_cost, account)
    

# Get number of shares purchased last 'buy' (the ammount most recently purchased, that hasnt been sold)
def num_to_sell(account):
    rename_later = 0
    
    for receipt in reversed(account['receipts']):
        rename_later += receipt[1]
        if rename_later > 0:
            return receipt[1]
    return 0 # only returns zero when all owned shares have been sold

# Returns the price per share in last transaction
def share_price_in_last_trans(account):
    total_amount = account['receipts'][-1][0]
    num_shares = account['receipts'][-1][1]
    
    share_price = abs(total_amount+5)/num_shares
    return share_price



In [59]:
# Simulator checks price per share every period(day), and acts according to the customizable conditions
# example strategy: buy at 5% loss, sell at 5% gain

def algo_trader(account, stats_sheet, percent):
    # Values to keep track of
    cost_two_days_ago = 0
    cost_one_day_ago = 0
    current_price = 0
    
    # Buys shares on first day (optional)
    initial = next(iter(stats_sheet.values()))
    buy_shares_by_percent_funds_owned(percent, initial, account)
    
    # For every period (currently period=1day)
    for cost in stats_sheet.values():   
        cost_two_days_ago = cost_one_day_ago
        cost_one_day_ago = current_price
        current_price = cost
        
        ################ Modify the buy/sell conditions below ################
        
        # example buy condition
        if (current_price <= 0.95*share_price_in_last_trans(account)) or (current_price < cost_one_day_ago and current_price < cost_two_days_ago): 
            buy_shares_by_percent_funds_owned(percent, current_price, account)            
        
        # example sell condition
        if (account['shares'] > 0) and (current_price >= 1.05*share_price_in_last_trans(account)): # wait until price is 5% up from the initial price, and sell
            sell(num_to_sell(account), current_price, account)  
            
        ################ Modify the buy/sell conditions above ################

In [60]:
## Initializing values ##

# Change example path to your preferred data sheet -- You can download data sheets from finance.yahoo.com
path = r"C:\Users\JohnDoe\AlgoTrader\data_sheets\TD_6months.csv"
stats_sheet = get_stats_from_excel(path)

trade_fee = 5 # trade_fee is the fee you pay per trade

percent = 0.2 # 'percent' is the value to use if you want to buy/sell by percent funds/shares owned

## Starting account info ##
# account = {'cash': initial amount, 'shares' : initial amount, receipts = [(delta cash, delta shares)]}
account = {'cash': 100000.00, 'shares' : 0, 'receipts': []}


In [61]:
## Execute Algorithm ##

algo_trader(account, stats_sheet, percent)

In [62]:
## Show Results ##

account

{'cash': 739.2414080000107,
 'shares': 1856,
 'receipts': [(-19999.879999999997, 381),
  (21651.04, -381),
  (-20328.75, 355),
  (-16258.32, 284),
  (-13001.0, 228),
  (-10393.91, 183),
  (11214.730183, -183),
  (13558.720456, -228),
  (17091.800284, -284),
  (21380.200709999997, -355),
  (-20979.5, 354),
  (-16766.599418, 291),
  (-13398.0, 236),
  (-10740.0, 190),
  (-8567.20031, 155),
  (-6873.7501250000005, 125),
  (-5509.5, 101),
  (5838.860100999999, -101),
  (7358.75, -125),
  (9215.95031, -155),
  (-8883.100308000001, 154),
  (-7072.500125, 125),
  (-5650.9699009999995, 99),
  (-4561.00008, 80),
  (-3630.600128, 64),
  (-2921.0, 54),
  (3068.14, -54),
  (3643.0, -64),
  (-3687.25013, 65),
  (-2930.5198960000002, 52),
  (-2354.900042, 42),
  (2536.4199160000003, -42),
  (3256.4400520000004, -52),
  (-3044.09, 51),
  (-2428.510041, 41),
  (2556.68, -41),
  (3281.4401020000005, -51),
  (4294.75013, -65),
  (5170.99976, -80),
  (6400.299703, -99),
  (-6262.699796, 102),
  (-5007.0,